#Part A

In [1]:
!pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp310-cp310-linux_x86_64.whl size=3365662 sha256=f1e066ed67d8e60643d756ff1d3a98bf9ae1eb8f261f997a8401ac3f59ba707a
  Stored in directory: /root/.cache/pip/wheels/e8/1b/b5/97ec4cfccdde26e0f3590ad6e09a5242d508dff09704ef86c1
Successfully built mpi4py


## Question 1

In [2]:
%%writefile question-1-mpi.py

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)
receivebuffer = numpy.zeros(1)
randNum[0] = numpy.random.random_sample(1)

if rank == 0:
        randNum0 = randNum[0]
        print("Process", rank, "drew the number", randNum0)
        comm.Send(randNum0, dest=1)
        comm.Recv(randNum, source=1)
        print("Process", rank, "has max value", randNum[0])

if rank == 1:
        randNum1 = randNum[0]
        print("Process", rank, "before receiving has the number", randNum1)
        comm.Recv(randNum, source=0)
        print("Process", rank, "received the number", randNum[0])
        
        if randNum1<randNum[0] :
          comm.Send(randNum, dest=0)
          print("Process", rank, "has max value", randNum[0]) 
        else:
          comm.Send(randNum1, dest=0)
          print("Process", rank, "has max value", randNum1)

Writing question-1-mpi.py


For Q1, we expect the following cell output to be of the form:


```
Process 0 drew the number 0.0177029648837449
Process 1 before receiving has the number 0.1308715233582639
Process 1 received the number 0.0177029648837449
Process 1 has max value 0.1308715233582639
Process 0 has max value 0.1308715233582639
```


(but with different numbers).

In [3]:
!mpiexec --allow-run-as-root --oversubscribe -n 2 python question-1-mpi.py

Process 0 drew the number 0.2319801925851811
Process 1 before receiving has the number 0.38177516551629687
Process 1 received the number 0.2319801925851811
Process 1 has max value 0.38177516551629687
Process 0 has max value 0.38177516551629687


## Question 2

In [4]:
%%writefile question-2-mpi.py

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

import time

randNum = numpy.zeros(1)
receivebuffer = numpy.zeros(1)
randNum[0] = numpy.random.random_sample(1)

if rank == 1:
        initNum = 0.0
        print("Process", rank, "before receiving has the number", initNum)
        
        randNum1 = randNum[0]
        print("Process", rank, "after generating sample", randNum1)
        req = comm.Irecv(randNum, source=0)
        req.Wait()
        if req.Test():
          print("Process", rank, "received the number", randNum[0])
        if randNum[0]<randNum1 :
          comm.Send(randNum1, dest=0)
          print("Process", rank, "has max value", randNum1) 
        else:
          comm.Send(randNum[0], dest=0)
          print("Process", rank, "has max value", randNum[0]) 
        
if rank == 0:
        randNum0 = randNum[0]
        print("Process", rank, "drew the number", randNum0)
        comm.Isend(randNum0, dest=1)
        req = comm.Irecv(randNum, source=1)
        req.Wait()
        print("Process", rank, "has max value", randNum[0]) 

Writing question-2-mpi.py


For Q2, we expect the following cell output to be in the form:

```
Process 1 before receiving has the number 0.0
Process 0 drew the number 0.714475096728165
Process 1 after generating sample 0.5177927276591507
Process 1 received the number 0.714475096728165
Process 1 has max value 0.714475096728165
Process 0 has max value 0.714475096728165
```
(but, again, with different numbers).


In [5]:
!mpiexec --allow-run-as-root --oversubscribe -n 2 python question-2-mpi.py

Process 0 drew the number 0.38072654943822215
Process 1 before receiving has the number 0.0
Process 1 after generating sample 0.6021943418553584
Process 1 received the number 0.38072654943822215
Process 1 has max value 0.6021943418553584
Process 0 has max value 0.6021943418553584


##Question 3

In [6]:
%%writefile question-3-mpi.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)
status = MPI.Status()

randNum[0] = numpy.random.random_sample(1)
print("Process", rank, "drew the number", randNum[0])      
max = comm.reduce(randNum, op=MPI.MAX, root=0)

if rank==0:
  print("Process", rank, "has max value", max[0]) 

Writing question-3-mpi.py


For Q3 (testing with 8 processes), we expect the following cell output to be in the form:

```
Process 3 drew the number 0.644414619435742
Process 7 drew the number 0.3318780441129008
Process 1 drew the number 0.30690199619345326
Process 2 drew the number 0.3000357589861674
Process 0 drew the number 0.37527495488187257
Process 4 drew the number 0.5049044705082459
Process 5 drew the number 0.748826873661416
Process 6 drew the number 0.44233981192412875
Process 0 has max value 0.748826873661416
```

(but with different numbers).



In [7]:
!mpiexec --allow-run-as-root --oversubscribe -n 8 python question-3-mpi.py

Process 2 drew the number 0.871631621952866
Process 1 drew the number 0.9407281474387121
Process 0 drew the number 0.16138676387161477
Process 3 drew the number 0.1703837185828293
Process 5 drew the number 0.9715362610494812
Process 6 drew the number 0.9247965205165188
Process 4 drew the number 0.7418947986475449
Process 7 drew the number 0.6369246534888999
Process 0 has max value 0.9715362610494812


##Question 4

In [8]:
%%writefile question-4-mpi.py

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

message = numpy.zeros(1)
status = MPI.Status()
randNum = numpy.zeros(1)

randNum[0] = numpy.random.random_sample(1)

if rank != 0:  
  print("Process", rank, "drew the number", randNum[0])
  comm.Send(randNum, dest=0)
if rank == 0:
  print("Process", rank, "drew the number", randNum[0])
  cur_max_num = randNum[0]
  cur_max_src = 0
  for i in range(comm.Get_size() - 1):
    comm.Recv(randNum, source=MPI.ANY_SOURCE, status=status)
    if randNum[0]>cur_max_num:
      cur_max_num, cur_max_src = randNum[0], status.source
  print("Process", rank, "max random number", cur_max_num, "found at rank", cur_max_src)         

Writing question-4-mpi.py


For Q4, we expect the following cell output to be in the form:


```
Process 0 drew the number 0.11288481815163798
Process 4 drew the number 0.060721653476019655
Process 3 drew the number 0.13311050208290287
Process 6 drew the number 0.6279276536062464
Process 5 drew the number 0.7787862146779945
Process 1 drew the number 0.17355707061903436
Process 2 drew the number 0.3926657952242526
Process 7 drew the number 0.3321183350590743
Process 0 max random number 0.7787862146779945 found at rank 5

```
(but with different numbers)


In [9]:
!mpiexec --allow-run-as-root --oversubscribe -n 8 python question-4-mpi.py

Process 4 drew the number 0.5673868192814495
Process 3 drew the number 0.36051256050899616
Process 6 drew the number 0.7898560204621764
Process 5 drew the number 0.705668128843396
Process 7 drew the number 0.4501368088386075
Process 1 drew the number 0.4599259922879838
Process 0 drew the number 0.2643321305786479
Process 2 drew the number 0.12789670404053188
Process 0 max random number 0.7898560204621764 found at rank 6


#Part B

##Question 1

before jit

In [10]:
import numpy as np
import numba
from numba import jit

def extract_bit(n, i):
  return n >> i & 1 == 1

print(extract_bit(0b1101, 0))
print(extract_bit(0b1101, 1))
print(extract_bit(0b1101, 2))
print(extract_bit(0b1101, 3))


True
False
True
True


In [11]:
def check_circuit(start, end):
  v = np.ones(32, dtype=np.int32)
  count = 0
  for bits in range(start, end):
    for i in range(32):
      v[i] = extract_bit(bits, i)

    if (((v[0] or v[1]) and (not v[1] or not v[3]) and (v[2] or v[3])
       and (not v[3] or not v[4]) and (v[4] or not v[5])
       and (v[5] or not v[6]) and (v[5] or v[6])
       and (v[6] or not v[15]) and (v[7] or not v[8])
       and (not v[7] or not v[13]) and (v[8] or v[9])
       and (v[8] or not v[9]) and (not v[9] or not v[10])
       and (v[9] or v[11]) and (v[10] or v[11])
       and (v[12] or v[13]) and (v[13] or not v[14])
       and (v[14] or v[15]))
       and
          ( (v[16] or v[17]) and (not v[17] or not v[19]) and (v[18] or v[19])
       and (not v[19] or not v[20]) and (v[20] or not v[21])
       and (v[21] or not v[22]) and (v[21] or v[22])
       and (v[22] or not v[31]) and (v[23] or not v[24])
       and (not v[23] or not v[29]) and (v[24] or v[25])
       and (v[24] or not v[25]) and (not v[25] or not v[26])
       and (v[25] or v[27]) and (v[26] or v[27])
       and (v[28] or v[29]) and (v[29] or not v[30])
       and (v[30] or v[31]))):
      count += 1
  return count


In [15]:
%time print(check_circuit(0, 1048576))


0
CPU times: user 10.6 s, sys: 11.1 ms, total: 10.6 s
Wall time: 10.8 s


after jit

In [16]:
from numba import jit,vectorize,int64,int32

@jit
def check_extract_bit(n, i):
  return n >> i & 1 == 1

@jit
def check_circuit_jit(start, end):
  v = np.ones(32, dtype=np.int32)
  count = 0
  for bits in range(start, end):
    for i in range(32):
      v[i] = check_extract_bit(bits, i)

    if (((v[0] or v[1]) and (not v[1] or not v[3]) and (v[2] or v[3])
       and (not v[3] or not v[4]) and (v[4] or not v[5])
       and (v[5] or not v[6]) and (v[5] or v[6])
       and (v[6] or not v[15]) and (v[7] or not v[8])
       and (not v[7] or not v[13]) and (v[8] or v[9])
       and (v[8] or not v[9]) and (not v[9] or not v[10])
       and (v[9] or v[11]) and (v[10] or v[11])
       and (v[12] or v[13]) and (v[13] or not v[14])
       and (v[14] or v[15]))
       and
          ( (v[16] or v[17]) and (not v[17] or not v[19]) and (v[18] or v[19])
       and (not v[19] or not v[20]) and (v[20] or not v[21])
       and (v[21] or not v[22]) and (v[21] or v[22])
       and (v[22] or not v[31]) and (v[23] or not v[24])
       and (not v[23] or not v[29]) and (v[24] or v[25])
       and (v[24] or not v[25]) and (not v[25] or not v[26])
       and (v[25] or v[27]) and (v[26] or v[27])
       and (v[28] or v[29]) and (v[29] or not v[30])
       and (v[30] or v[31]))):
      count += 1
  return count


In [22]:
%time print(check_circuit_jit(0, 1048576)) # notice the function name change here
%time print(check_circuit_jit(0, 4294967296)) # full 32-bit test.


0
CPU times: user 32.1 ms, sys: 1e+03 ns, total: 32.1 ms
Wall time: 32.1 ms
81
CPU times: user 2min 9s, sys: 361 ms, total: 2min 9s
Wall time: 2min 9s


##Question 2

In [18]:
import os
os.environ['NUMBAPRO_NVVM']='/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so'  
os.environ['NUMBAPRO_LIBDEVICE']='/usr/local/cuda-10.0/nvvm/libdevice/'
os.environ['NUMBAPRO_CUDA_DRIVER']='/usr/local/cuda-10.0/nvvm/libcuda.so'

import numpy as np
import numba
from numba import cuda, jit

@cuda.jit
def extract_bit_jit(n, i):
  return n >> i & 1 == 1

@cuda.jit
def check_circuit_gpu(start, end, count_out):
  v = cuda.local.array(32, dtype=numba.int32)
  id = cuda.grid(1)
  per = (end - start) // cuda.gridsize(1)
  thread_start = id * per
  thread_end = (id+1) * per
  count = 0
  for bits in range(thread_start , thread_end):
    for i in range(32):
      v[i] = extract_bit_jit(bits, i)

    if (((v[0] or v[1]) and (not v[1] or not v[3]) and (v[2] or v[3])
       and (not v[3] or not v[4]) and (v[4] or not v[5])
       and (v[5] or not v[6]) and (v[5] or v[6])
       and (v[6] or not v[15]) and (v[7] or not v[8])
       and (not v[7] or not v[13]) and (v[8] or v[9])
       and (v[8] or not v[9]) and (not v[9] or not v[10])
       and (v[9] or v[11]) and (v[10] or v[11])
       and (v[12] or v[13]) and (v[13] or not v[14])
       and (v[14] or v[15]))
       and
          ( (v[16] or v[17]) and (not v[17] or not v[19]) and (v[18] or v[19])
       and (not v[19] or not v[20]) and (v[20] or not v[21])
       and (v[21] or not v[22]) and (v[21] or v[22])
       and (v[22] or not v[31]) and (v[23] or not v[24])
       and (not v[23] or not v[29]) and (v[24] or v[25])
       and (v[24] or not v[25]) and (not v[25] or not v[26])
       and (v[25] or v[27]) and (v[26] or v[27])
       and (v[28] or v[29]) and (v[29] or not v[30])
       and (v[30] or v[31]))):
      count += 1
  cuda.atomic.add(count_out, 0, count)
  



In [19]:
# part b
#16384 threads should take around 52 ms, 1 will take ~5 minutes
count_out = cuda.to_device(np.zeros(shape=1, dtype=np.int32))
check_circuit_gpu[128, 128](0, 4294967296, count_out)
%time print(count_out.copy_to_host(), 'solutions')

[81] solutions
CPU times: user 153 ms, sys: 0 ns, total: 153 ms
Wall time: 155 ms


In [20]:
#part c
block_list= [1, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]
for block in block_list:
  count_out = cuda.to_device(np.zeros(shape=1, dtype=np.int32))
  check_circuit_gpu[block, 128](0, 4294967296, count_out)
  %time print(f"number of blocks: {block} found {count_out.copy_to_host()} solutions")

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


number of blocks: 1 found [81] solutions
CPU times: user 2.75 s, sys: 4.98 ms, total: 2.75 s
Wall time: 2.74 s


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 8 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


number of blocks: 8 found [81] solutions
CPU times: user 325 ms, sys: 4 µs, total: 325 ms
Wall time: 322 ms
number of blocks: 16 found [81] solutions
CPU times: user 161 ms, sys: 2 µs, total: 161 ms
Wall time: 161 ms


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 16 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 32 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


number of blocks: 32 found [81] solutions
CPU times: user 81.2 ms, sys: 1 ms, total: 82.2 ms
Wall time: 80.2 ms
number of blocks: 64 found [81] solutions
CPU times: user 65.4 ms, sys: 0 ns, total: 65.4 ms
Wall time: 65.3 ms
number of blocks: 128 found [81] solutions
CPU times: user 62.4 ms, sys: 0 ns, total: 62.4 ms
Wall time: 62.3 ms
number of blocks: 256 found [81] solutions
CPU times: user 53.1 ms, sys: 0 ns, total: 53.1 ms
Wall time: 53.1 ms


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 64 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


number of blocks: 512 found [81] solutions
CPU times: user 49.7 ms, sys: 0 ns, total: 49.7 ms
Wall time: 48.3 ms
number of blocks: 1024 found [81] solutions
CPU times: user 48.5 ms, sys: 0 ns, total: 48.5 ms
Wall time: 48.3 ms
number of blocks: 2048 found [81] solutions
CPU times: user 48 ms, sys: 0 ns, total: 48 ms
Wall time: 48 ms
number of blocks: 4096 found [81] solutions
CPU times: user 48.3 ms, sys: 3 µs, total: 48.3 ms
Wall time: 48.1 ms


##Question 3

In [21]:
import numpy as np
import numba
from numba import cuda, jit
import os
os.environ['NUMBAPRO_NVVM']='/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so'  
os.environ['NUMBAPRO_LIBDEVICE']='/usr/local/cuda-10.0/nvvm/libdevice'

@jit
def median(a, b, c):
  if a > b:
    if b > c:
      return b 
    elif a > c: 
      return c 
    else:
      return a  
  else: 
    if a > c:
      return a
    elif b > c:
      return c
    else:
      return b 

@jit
def median_filter_seq(A):
  n = np.shape(A)[0]
  r = np.zeros(n, dtype=np.uint8)
  r[0] = A[0]
  r[n-1] = A[n-1]
  
  for i in range(1,n-1):
    r[i] = median(A[i-1], A[i], A[i+1])
  
  return r

A = np.random.randint(256, size=256, dtype=np.uint8)
B = np.array([1, 5, 3, 4, 2, 9, 4])

A_seq = median_filter_seq(A)
B_seq = median_filter_seq(B)

@cuda.jit
def median_filter(A):
  i = cuda.grid(1)

  if (0 < i < A.shape[0] - 1):
    median_val = median(A[i - 1], A[i], A[i + 1])
    A[i] = median_val

median_filter[1,256](A)
median_filter[1,7](B)

assert np.array_equal(B, B_seq)
assert np.array_equal(A, A_seq)



/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
